In [17]:
!pip install netCDF4 cftime


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
# Merged + hardened script: open NetCDF (with fallbacks) -> extract year -> clip by AOI -> reproj -> save GeoTIFF
# Requirements: xarray, rioxarray, geopandas, numpy, rasterio (for writing), netCDF4 or h5py (for fallback)
# pip install xarray rioxarray geopandas rasterio netCDF4 h5py cftime

import os
import numpy as np
import xarray as xr

# optional fallbacks
try:
    import netCDF4 as nc4
except Exception:
    nc4 = None

try:
    import h5py
except Exception:
    h5py = None

# spatial libs
import rioxarray
import geopandas as gpd

# === USER INPUTS ===
netcdf_path = r"D:\Cognitud\Id\timeseries-id-annual-mean_cmip6-x0.25_ensemble-all-ssp585_timeseries-smooth_median_2015-2100.nc"
aoi_shapefile = r"D:\Cognitud\Worldshape file\aoi_world.shp"
output_folder = r"D:\Cognitud\Id\8.5_2100"
year_to_extract = 2100  # integer year to extract
output_filename = None  # or "my_output.tif"

os.makedirs(output_folder, exist_ok=True)


def try_open_dataset(path):
    """Try xarray engines first, then fall back to netCDF4/h5py to construct an xarray.Dataset."""
    engines_to_try = ["netcdf4", "h5netcdf", "scipy"]
    for eng in engines_to_try:
        try:
            print(f"Trying xarray.open_dataset(..., engine='{eng}')")
            ds = xr.open_dataset(path, engine=eng, decode_times=True)
            print(f"Opened with engine='{eng}'")
            return ds
        except ValueError as ve:
            print(f"Engine '{eng}' failed: {ve}")
        except Exception as e:
            print(f"xarray with engine '{eng}' raised: {e}")

    # netCDF4 fallback
    if nc4 is not None:
        try:
            print("Falling back to netCDF4 library to read the file...")
            ds_nc = nc4.Dataset(path, mode="r")

            # heuristic: find first variable with >=2 dims (likely the raster variable)
            data_var_name = None
            for vn in ds_nc.variables:
                v = ds_nc.variables[vn]
                if getattr(v, "dimensions", None) and len(v.dimensions) >= 2:
                    data_var_name = vn
                    break
            if data_var_name is None:
                for vn in ds_nc.variables:
                    v = ds_nc.variables[vn]
                    if getattr(v, "ndim", 0) > 0:
                        data_var_name = vn
                        break
            if data_var_name is None:
                raise RuntimeError("Could not identify a data variable in NetCDF via netCDF4.")

            print(f"Detected data variable: {data_var_name}")
            var = ds_nc.variables[data_var_name]
            data = var[:].astype(np.float32)

            # build coords
            coords = {}
            for dim in var.dimensions:
                if dim in ds_nc.variables:
                    coords[dim] = ds_nc.variables[dim][:]
                else:
                    coords[dim] = np.arange(data.shape[var.dimensions.index(dim)])

            # time conversion if present
            if "time" in ds_nc.variables:
                try:
                    time_var = ds_nc.variables["time"]
                    units = getattr(time_var, "units", None)
                    cal = getattr(time_var, "calendar", "standard")
                    times = nc4.num2date(time_var[:], units=units, calendar=cal)
                    coords["time"] = np.array(times)
                except Exception:
                    coords["time"] = ds_nc.variables["time"][:]

            # lat/lon mapping heuristics
            lat_name = None
            lon_name = None
            for cand in ("lat", "latitude", "y"):
                if cand in ds_nc.variables:
                    lat_name = cand
                    break
            for cand in ("lon", "longitude", "x"):
                if cand in ds_nc.variables:
                    lon_name = cand
                    break
            if lat_name and lon_name:
                coords["lat"] = ds_nc.variables[lat_name][:]
                coords["lon"] = ds_nc.variables[lon_name][:]
            else:
                # try mapping dims names if they match lat/lon-like
                for d in var.dimensions:
                    if d.lower() in ("lat", "latitude", "y"):
                        coords["lat"] = ds_nc.variables.get(d, np.arange(data.shape[var.dimensions.index(d)]))[:]
                    if d.lower() in ("lon", "longitude", "x"):
                        coords["lon"] = ds_nc.variables.get(d, np.arange(data.shape[var.dimensions.index(d)]))[:]

            # prepare xarray coords only for 1D arrays
            xr_coords = {}
            for k, v in coords.items():
                try:
                    arr = np.array(v)
                    if arr.ndim == 1:
                        xr_coords[k] = arr
                except Exception:
                    pass

            ds_xr = xr.Dataset({data_var_name: (var.dimensions, data)}, coords=xr_coords)
            ds_xr.attrs.update({k: getattr(ds_nc, k) for k in ds_nc.ncattrs()})
            ds_nc.close()
            print("Successfully built xarray.Dataset from netCDF4.")
            return ds_xr
        except Exception as e:
            print("netCDF4 fallback failed:", e)

    # h5py fallback
    if h5py is not None:
        try:
            print("Falling back to h5py to inspect the file...")
            hf = h5py.File(path, "r")
            data_name = None

            def find_dataset(group):
                nonlocal data_name
                for k, v in group.items():
                    if isinstance(v, h5py.Dataset):
                        if v.ndim >= 2 and data_name is None:
                            data_name = v.name
                            return
                    elif isinstance(v, h5py.Group):
                        find_dataset(v)

            find_dataset(hf)
            if data_name is None:
                raise RuntimeError("Could not find a suitable dataset in the HDF5 file using h5py.")
            arr = hf[data_name][()]
            dims = tuple(f"dim_{i}" for i in range(arr.ndim))
            ds_xr = xr.Dataset({os.path.basename(data_name): (dims, arr)})
            hf.close()
            print("Built dataset via h5py fallback.")
            return ds_xr
        except Exception as e:
            print("h5py fallback failed:", e)

    raise RuntimeError(
        "xarray could not find a usable engine and neither netCDF4 nor h5py are available. "
        "Install netCDF4/cftime or h5py (pip or conda) and retry."
    )


# === OPEN DATASET ===
ds = try_open_dataset(netcdf_path)

# Summary
print("\n📦 Variables in dataset:")
for v in ds.data_vars:
    print(f" - {v}: dtype={ds[v].dtype}, dims={ds[v].dims}")

if len(ds.data_vars) == 0:
    raise ValueError("No data variables found in NetCDF.")

var_name = list(ds.data_vars.keys())[0]
print(f"\n✅ Using variable: {var_name}")
da = ds[var_name]


# === SELECT YEAR ===
def select_year_da(da, year):
    if "time" in da.dims:
        try:
            import pandas as pd
            tvals = pd.to_datetime(da.coords["time"].values)
            matches = [i for i, t in enumerate(tvals) if getattr(t, "year", None) == int(year)]
            if matches:
                print(f"Exact match for year {year} at index {matches[0]} -> {tvals[matches[0]]}")
                return da.isel(time=matches[0])
            years = np.array([getattr(t, "year", np.nan) for t in tvals], dtype=float)
            if np.all(np.isnan(years)):
                raise ValueError("Could not extract year from time coordinate.")
            nearest = int(np.nanargmin(np.abs(years - year)))
            print(f"No exact year. Using nearest index {nearest} (year {int(years[nearest])}) -> {tvals[nearest]}")
            return da.isel(time=nearest)
        except Exception as e:
            for i, t in enumerate(da.coords["time"].values):
                if str(year) in str(t):
                    print(f"String matched year in time value: index {i} -> {t}")
                    return da.isel(time=i)
            raise RuntimeError(f"Could not select year {year} from time coordinate: {e}")
    else:
        print("No 'time' dimension present; using DataArray as-is.")
        return da


da_sel = select_year_da(da, year_to_extract)


# === SPATIAL DIM NAMES & SQUEEZE ===
# rename if necessary
rename_map = {}
if "latitude" in da_sel.dims and "longitude" in da_sel.dims:
    rename_map = {"latitude": "lat", "longitude": "lon"}
elif "lat" in da_sel.dims and "lon" in da_sel.dims:
    rename_map = {}
elif "y" in da_sel.dims and "x" in da_sel.dims:
    rename_map = {}

if rename_map:
    da_sel = da_sel.rename(rename_map)
    print("Renamed dimensions:", rename_map)

# squeeze and slice non-spatial dims
da_2d = da_sel.squeeze(drop=True)
spatial = ("lat", "lon", "y", "x")
non_spatial = [d for d in da_2d.dims if d not in spatial]
if non_spatial:
    sel = {d: 0 for d in non_spatial}
    da_2d = da_2d.isel(sel)
    print("Sliced non-spatial dims at index 0:", non_spatial)


# set spatial dims for rioxarray
if ("lat" in da_2d.coords) and ("lon" in da_2d.coords):
    da_2d = da_2d.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
elif ("y" in da_2d.coords) and ("x" in da_2d.coords):
    da_2d = da_2d.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
else:
    raise RuntimeError("Could not find lat/lon or x/y coordinates in DataArray. Inspect da.coords keys: " + ", ".join(map(str, da_2d.coords.keys())))


# === WRITE CRS (assume EPSG:4326 if missing) ===
if not da_2d.rio.crs:
    print("Writing CRS EPSG:4326 to DataArray.")
    da_2d = da_2d.rio.write_crs("EPSG:4326", inplace=False)
else:
    print("DataArray has CRS:", da_2d.rio.crs)


# === LOAD AOI and MATCH CRS ===
aoi = gpd.read_file(aoi_shapefile)
print("AOI features:", len(aoi))
if aoi.crs is None:
    aoi = aoi.set_crs("EPSG:4326")
elif aoi.crs.to_string() != "EPSG:4326":
    aoi = aoi.to_crs("EPSG:4326")
    print("Reprojected AOI to EPSG:4326")


# === CLIP ===
print("Clipping raster with AOI (may take time for large rasters)...")
da_clipped = da_2d.rio.clip(aoi.geometry, aoi.crs, drop=True, invert=False)


# === REPROJECT to EPSG:3857 ===
print("Reprojecting to EPSG:3857...")
da_reproj = da_clipped.rio.reproject("EPSG:3857")


# === OUTPUT PATH ===
if output_filename:
    output_tif = os.path.join(output_folder, output_filename)
else:
    output_tif = os.path.join(output_folder, f"{var_name}_{year_to_extract}_clipped_3857.tif")

# dtype / convert
out_dtype = np.float32 if np.issubdtype(da_reproj.dtype, np.floating) else da_reproj.dtype
da_reproj = da_reproj.astype(out_dtype)

# dataset for export
ds_out = da_reproj.to_dataset(name=var_name)

print("Writing GeoTIFF to:", output_tif)
try:
    ds_out.rio.to_raster(output_tif, driver="GTiff", dtype=out_dtype)
    print("✅ GeoTIFF saved:", output_tif)
except Exception as e:
    print("Failed to write GeoTIFF directly:", e)
    try:
        import rasterio
        print("Attempting rasterio fallback via rioxarray...")
        ds_out.rio.to_raster(output_tif)
        print("✅ GeoTIFF saved via fallback:", output_tif)
    except Exception as e2:
        raise RuntimeError("All write attempts failed. Last error: " + str(e2))

# === BASIC STATS ===
da_stats = da_reproj
try:
    vmin = float(da_stats.min().values)
    vmax = float(da_stats.max().values)
    vmean = float(da_stats.mean().values)
    nancount = int(np.isnan(da_stats.values).sum())
    print("Output stats -- min, max, mean, nancount:")
    print(vmin, vmax, vmean, nancount)
except Exception as e:
    print("Could not compute stats:", e)

print("Done.")


Trying xarray.open_dataset(..., engine='netcdf4')
Engine 'netcdf4' failed: unrecognized engine 'netcdf4' must be one of your download engines: ['rasterio', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
Trying xarray.open_dataset(..., engine='h5netcdf')
Engine 'h5netcdf' failed: unrecognized engine 'h5netcdf' must be one of your download engines: ['rasterio', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
Trying xarray.open_dataset(..., engine='scipy')
Engine 'scipy' failed: unrecognized engine 'scipy' must be one of your download engines: ['rasterio', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
Fall

In [23]:
import os
import xarray as xr
import rioxarray

# ----------------------------------
# 1. Input NetCDF file
# ----------------------------------
nc_file = r"D:\Landslide paper\03_IPED_RES_0P10\03_IPED_RES_0P10\IPED_Mean\IPED_mean_1991.nc"

# ----------------------------------
# 2. Output folder
# ----------------------------------
output_folder = r"D:\Landslide paper\IPED_1991_TIFFS"
os.makedirs(output_folder, exist_ok=True)

# ----------------------------------
# 3. Open dataset
# ----------------------------------
ds = xr.open_dataset(nc_file)

print("Dataset structure:")
print(ds)

# ----------------------------------
# 4. Select precipitation variable
# ----------------------------------
variable_name = "pcp"   # change if different
data = ds[variable_name]

# ----------------------------------
# 5. Set CRS (IPED is lat/lon WGS84)
# ----------------------------------
data = data.rio.write_crs("EPSG:4326")

# ----------------------------------
# 6. Export each day as GeoTIFF
# ----------------------------------
for i in range(len(data.time)):
    
    single_day = data.isel(time=i)
    
    # Format date
    date_str = str(single_day.time.values)[:10]
    
    output_path = os.path.join(
        output_folder,
        f"IPED_2023_{date_str}.tif"
    )
    
    single_day.rio.to_raster(output_path)
    
    print(f"Saved: {output_path}")

print("\nAll 2023 daily rainfall TIFFs exported successfully!")


Dataset structure:
<xarray.Dataset> Size: 130MB
Dimensions:      (lat: 304, lon: 293, time: 365)
Coordinates:
  * lat          (lat) float64 2kB 6.8 6.9 7.0 7.1 7.2 ... 36.8 36.9 37.0 37.1
  * lon          (lon) float64 2kB 68.2 68.3 68.4 68.5 ... 97.1 97.2 97.3 97.4
  * time         (time) datetime64[ns] 3kB 1991-01-01 1991-01-02 ... 1991-12-31
Data variables:
    spatial_ref  int64 8B ...
    pcp          (lat, lon, time) float32 130MB ...
Attributes:
    description:             Indian Precipitation Ensemble Dataset (IPED)
    principal investigator:  Dr. Manabendra Saharia (msaharia@iitd.ac.in)
    institution:             HydroSense Lab, IIT Delhi and Indian Meteorologi...
    contributors:            Anagha P, Manabendra Saharia, Sreejith O P
    website:                 https://hydrosense.iitd.ac.in/
    dx:                      0.1
    dy:                      0.1
Saved: D:\Landslide paper\IPED_1991_TIFFS\IPED_2023_1991-01-01.tif
Saved: D:\Landslide paper\IPED_1991_TIFFS\IPED_2

In [24]:
import os
import glob
import rasterio
import numpy as np

# ---------------------------------------------------
# 1. Base Directory (where all yearly folders exist)
# ---------------------------------------------------
base_dir = r"D:\Landslide paper\03_IPED_RES_0P10\Mean"

# ---------------------------------------------------
# 2. Output Folder for Annual Rainfall Maps
# ---------------------------------------------------
output_dir = os.path.join(base_dir, "Annual_Rainfall_Maps")
os.makedirs(output_dir, exist_ok=True)

# ---------------------------------------------------
# 3. Loop Through Years (1991–2023)
# ---------------------------------------------------
for year in range(1991, 2024):
    
    print(f"\nProcessing Year: {year}")
    
    year_folder = os.path.join(base_dir, f"IPED_{year}_TIFFS")
    
    if not os.path.exists(year_folder):
        print(f"Folder not found for {year}, skipping...")
        continue
    
    tif_files = sorted(glob.glob(os.path.join(year_folder, "*.tif")))
    
    if len(tif_files) == 0:
        print(f"No TIFF files found for {year}, skipping...")
        continue
    
    print(f"Total daily files: {len(tif_files)}")
    
    # Read first file for metadata
    with rasterio.open(tif_files[0]) as src:
        meta = src.meta.copy()
        annual_sum = np.zeros((src.height, src.width), dtype=np.float32)
    
    # Sum daily rainfall
    for file in tif_files:
        with rasterio.open(file) as src:
            data = src.read(1)
            
            # Replace nodata with 0
            if src.nodata is not None:
                data = np.where(data == src.nodata, 0, data)
            
            annual_sum += data
    
    # Update metadata
    meta.update(dtype=rasterio.float32)
    
    # Save output
    output_file = os.path.join(output_dir, f"Annual_Rainfall_{year}.tif")
    
    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(annual_sum.astype(rasterio.float32), 1)
    
    print(f"Saved: {output_file}")

print("\n✅ All years processed successfully!")



Processing Year: 1991
Total daily files: 365
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1991.tif

Processing Year: 1992
Total daily files: 366
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1992.tif

Processing Year: 1993
Total daily files: 365
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1993.tif

Processing Year: 1994
Total daily files: 365
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1994.tif

Processing Year: 1995
Total daily files: 365
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1995.tif

Processing Year: 1996
Total daily files: 366
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1996.tif

Processing Year: 1997
Total daily files: 365
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Annual_Rainfall_1997.tif

Processing Year: 19

In [25]:
import os
import glob
import rasterio
import numpy as np

# ---------------------------------------------------
# 1. Directory where Annual Rainfall Maps are stored
# ---------------------------------------------------
annual_dir = r"D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps"

# ---------------------------------------------------
# 2. Output Directory for Decadal Maps
# ---------------------------------------------------
output_dir = os.path.join(annual_dir, "Decadal_Mean_Maps")
os.makedirs(output_dir, exist_ok=True)

# ---------------------------------------------------
# 3. Define Decades
# ---------------------------------------------------
decades = [
    (1993, 2003),
    (2003, 2013),
    (2013, 2023),
]

# ---------------------------------------------------
# 4. Loop Through Each Decade
# ---------------------------------------------------
for start_year, end_year in decades:
    
    print(f"\nProcessing Decade: {start_year}-{end_year}")
    
    files = []
    for year in range(start_year, end_year + 1):
        file_path = os.path.join(annual_dir, f"Annual_Rainfall_{year}.tif")
        if os.path.exists(file_path):
            files.append(file_path)
    
    if len(files) == 0:
        print("No files found for this decade.")
        continue
    
    print(f"Number of annual rasters used: {len(files)}")
    
    # Read first file for metadata
    with rasterio.open(files[0]) as src:
        meta = src.meta.copy()
        decade_sum = np.zeros((src.height, src.width), dtype=np.float32)
    
    # Sum annual rasters
    for file in files:
        with rasterio.open(file) as src:
            data = src.read(1)
            decade_sum += data
    
    # Calculate mean
    decade_mean = decade_sum / len(files)
    
    # Update metadata
    meta.update(dtype=rasterio.float32)
    
    # Save output
    output_file = os.path.join(output_dir, 
                               f"Decadal_Mean_{start_year}_{end_year}.tif")
    
    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(decade_mean.astype(rasterio.float32), 1)
    
    print(f"Saved: {output_file}")

# ---------------------------------------------------
# 5. Long-Term Mean (1991–2023)
# ---------------------------------------------------
print("\nProcessing Long-Term Mean (1991–2023)")

all_files = glob.glob(os.path.join(annual_dir, "Annual_Rainfall_*.tif"))

with rasterio.open(all_files[0]) as src:
    meta = src.meta.copy()
    total_sum = np.zeros((src.height, src.width), dtype=np.float32)

for file in all_files:
    with rasterio.open(file) as src:
        data = src.read(1)
        total_sum += data

long_term_mean = total_sum / len(all_files)

meta.update(dtype=rasterio.float32)

output_file = os.path.join(output_dir, "Long_Term_Mean_1991_2023.tif")

with rasterio.open(output_file, "w", **meta) as dst:
    dst.write(long_term_mean.astype(rasterio.float32), 1)

print("✅ All Decadal Maps Created Successfully!")



Processing Decade: 1993-2003
Number of annual rasters used: 11
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Decadal_Mean_Maps\Decadal_Mean_1993_2003.tif

Processing Decade: 2003-2013
Number of annual rasters used: 11
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Decadal_Mean_Maps\Decadal_Mean_2003_2013.tif

Processing Decade: 2013-2023
Number of annual rasters used: 11
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Rainfall_Maps\Decadal_Mean_Maps\Decadal_Mean_2013_2023.tif

Processing Long-Term Mean (1991–2023)
✅ All Decadal Maps Created Successfully!


In [26]:
import os
import glob
import rasterio
import numpy as np

base_dir = r"D:\Landslide paper\03_IPED_RES_0P10\Mean"
output_dir = os.path.join(base_dir, "Annual_Max_Intensity")
os.makedirs(output_dir, exist_ok=True)

for year in range(1991, 2024):
    
    print(f"\nProcessing Max Intensity for {year}")
    
    year_folder = os.path.join(base_dir, f"IPED_{year}_TIFFS")
    
    if not os.path.exists(year_folder):
        print("Folder not found, skipping...")
        continue
    
    tif_files = sorted(glob.glob(os.path.join(year_folder, "*.tif")))
    
    if len(tif_files) == 0:
        print("No TIFF files found.")
        continue
    
    with rasterio.open(tif_files[0]) as src:
        meta = src.meta.copy()
        max_rain = src.read(1).astype(np.float32)
    
    for file in tif_files[1:]:
        with rasterio.open(file) as src:
            data = src.read(1).astype(np.float32)
            max_rain = np.maximum(max_rain, data)
    
    meta.update(dtype=rasterio.float32)
    
    output_file = os.path.join(output_dir, f"Max_Daily_Rainfall_{year}.tif")
    
    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(max_rain, 1)
    
    print(f"Saved: {output_file}")

print("\n✅ Annual Intensity Maps Created Successfully!")



Processing Max Intensity for 1991
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1991.tif

Processing Max Intensity for 1992
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1992.tif

Processing Max Intensity for 1993
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1993.tif

Processing Max Intensity for 1994
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1994.tif

Processing Max Intensity for 1995
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1995.tif

Processing Max Intensity for 1996
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1996.tif

Processing Max Intensity for 1997
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Max_Daily_Rainfall_1997.tif

Processing Max Intensity for 1998
Saved: D:\Landslide paper\03_IPED_RES_0P1

In [28]:
import os
import glob
import rasterio
import numpy as np
annual_intensity_dir = r"D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity"
decadal_output = os.path.join(annual_intensity_dir, "Decadal_Intensity")
os.makedirs(decadal_output, exist_ok=True)

decades = [
    (1993, 2003),
    (2003, 2013),
    (2013, 2023),
]

for start, end in decades:
    
    print(f"\nProcessing Decade {start}-{end}")
    
    files = []
    for year in range(start, end+1):
        file_path = os.path.join(annual_intensity_dir, f"Max_Daily_Rainfall_{year}.tif")
        if os.path.exists(file_path):
            files.append(file_path)
    
    if len(files) == 0:
        print("No files found.")
        continue
    
    with rasterio.open(files[0]) as src:
        meta = src.meta.copy()
        sum_data = np.zeros((src.height, src.width), dtype=np.float32)
    
    for file in files:
        with rasterio.open(file) as src:
            data = src.read(1)
            sum_data += data
    
    decade_mean = sum_data / len(files)
    
    meta.update(dtype=rasterio.float32)
    
    output_file = os.path.join(decadal_output, 
                               f"Decadal_Max_Intensity_{start}_{end}.tif")
    
    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(decade_mean.astype(np.float32), 1)
    
    print(f"Saved: {output_file}")

print("\n✅ Decadal Intensity Maps Created Successfully!")


Processing Decade 1993-2003
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Decadal_Intensity\Decadal_Max_Intensity_1993_2003.tif

Processing Decade 2003-2013
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Decadal_Intensity\Decadal_Max_Intensity_2003_2013.tif

Processing Decade 2013-2023
Saved: D:\Landslide paper\03_IPED_RES_0P10\Mean\Annual_Max_Intensity\Decadal_Intensity\Decadal_Max_Intensity_2013_2023.tif

✅ Decadal Intensity Maps Created Successfully!
